In [2]:
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd

In [ ]:
datasetEstaciones = pd.read_excel('./ListadoEstaciones-20190206.xlsx')
datasetIncendios = pd.read_csv('./modis_2021_Spain_incendios.csv')
datasetIncendios = datasetIncendios.iloc[:, [0,1,2,5]]

aux = 0
for di in range(len(datasetIncendios)):
    print(f"iteration {di} with date {datasetIncendios.iloc[di,3]}")
    if aux != datasetIncendios.iloc[di,3]:
        print(f"making Aemet2021-{datasetIncendios.iloc[di,3][5:]}.csv")
        datasetClima = pd.read_excel(f'./ClimaAemet2021/Aemet2021-{datasetIncendios.iloc[di,3][5:7]}/Aemet2021-{datasetIncendios.iloc[di,3][5:]}.xls')
        datasetClima = datasetClima.iloc[4:,:]
        datasetClima = datasetClima.assign(latitude =  np.zeros(len(datasetClima)))
        datasetClima = datasetClima.assign(longitude = np.zeros(len(datasetClima)))
        for dc in range(len(datasetClima)):
            for de in range(len(datasetEstaciones)):
                if datasetClima.iloc[dc,0].upper() == datasetEstaciones.iloc[de,2].upper():
                    if datasetEstaciones.iloc[de,5][-1] == 'W':
                        datasetClima.iloc[dc,-2] = int(datasetEstaciones.iloc[de,4][:2])+int(datasetEstaciones.iloc[de,4][2:4])/60 + int(datasetEstaciones.iloc[de,4][4:6])/3600
                        datasetClima.iloc[dc,-1] = (int(datasetEstaciones.iloc[de,5][:2])+int(datasetEstaciones.iloc[de,5][2:4])/60 + int(datasetEstaciones.iloc[de,5][4:6])/3600)*-1
                    else:
                        datasetClima.iloc[dc,-2] = int(datasetEstaciones.iloc[de,4][:2])+int(datasetEstaciones.iloc[de,4][2:4])/60 + int(datasetEstaciones.iloc[de,4][4:6])/3600
                        datasetClima.iloc[dc,-1] = (int(datasetEstaciones.iloc[de,5][:2])+int(datasetEstaciones.iloc[de,5][2:4])/60 + int(datasetEstaciones.iloc[de,5][4:6])/3600)
                    break
                else:
                    try:
                        geolocator = Nominatim(user_agent="zaragoza") 
                        print(datasetClima.iloc[dc,0])
                        location = geolocator.geocode(datasetClima.iloc[dc,0])
                        print(location.latitude, location.longitude)
                        datasetClima.iloc[dc,-2] = location.latitude
                        datasetClima.iloc[dc,-1] = location.longitude
                    except:
                        print("ERROR")
                        datasetClima.iloc[dc,-2] = None
                        datasetClima.iloc[dc,-1] = None
                    finally:
                        break
        datasetClima = datasetClima.dropna()

        datasetClima.columns = ['Estación', 'Provincia', 'Temperatura máxima (ºC)', 'Temperatura mínima (ºC)', 'Temperatura media (ºC)', 'Racha (km/h)', 
        'Velocidad máxima (km/h)', 'Precipitación 00-24h (mm)','Precipitación 00-06h (mm)', 'Precipitación 06-12h (mm)', 'Precipitación 12-18h (mm)' ,'Precipitación 18-24h (mm)', 'Latitude', 'Longitude']

        datasetClima.to_csv(f'ClimaData/Aemet2021-{datasetIncendios.iloc[di,3][5:]}.csv',index=False)
    aux = datasetIncendios.iloc[di,3]

In [3]:

geolocator = Nominatim(user_agent="dieo", ) 
location = geolocator.geocode("Estaca de Bares")
print(location.latitude, location.longitude)

43.7903123 -7.6882854
